In [5]:
import requests
from datetime import datetime
import pandas as pd
import time
from google.cloud import bigquery
import os

In [ ]:
credentials_path = 'chainlink-455803-8d7e35b14f53.json'

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [315]:
chains = ['Bitcoin', 'Ethereum', 'Solana', 'Cardano', 'Ripple']

In [7]:
current_timestamp = datetime.now()

In [ ]:
client = bigquery.Client()

## Total Value Locked

In [318]:
tvl_formatted_json_response = []
for chain in chains:
    url = 'https://api.llama.fi/v2/historicalChainTvl/{chain}'.format(chain=chain)
    response = requests.get(url)
    for row in response.json():
        new_row = {}
        new_row['EXTRACT_TIME'] = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")
        new_row['BLOCKCHAIN'] = chain.upper()
        new_row['DATE'] = datetime.fromtimestamp(row['date']).strftime("%Y-%m-%d")
        new_row['TVL'] = row['tvl']
        tvl_formatted_json_response += [new_row]

In [ ]:
errors = client.insert_rows_json(tvl_table_id, tvl_formatted_json_response)

In [ ]:
if errors:
    print("Ran into errors uploading!")

## Total Stablecoin Marketcap

In [327]:
stablecoin_marketcap_formatted_json_response = []
for chain in chains:
    if chain.upper() != "BITCOIN":
        url = "https://stablecoins.llama.fi/stablecoincharts/{chain}".format(chain=chain)
        response = requests.get(url)
        for row in response.json():
            new_row = {}
            new_row['EXTRACT_TIME'] = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")
            new_row['BLOCKCHAIN'] = chain.upper()
            new_row['DATE'] = datetime.fromtimestamp(int(row['date'])).strftime("%Y-%m-%d")
            new_row['STABLECOIN_MARKETCAP'] = int(row['totalCirculating']['peggedUSD'])
            stablecoin_marketcap_formatted_json_response += [new_row]

In [ ]:
errors = client.insert_rows_json(stablecoin_marketcap_table_id, stablecoin_marketcap_formatted_json_response)

In [ ]:
if errors:
    print("Ran into errors uploading!")

## Total Market Cap and Volume

In [336]:
marketcap_formatted_json_response = []
volume_formatted_json_response = []
currency = 'usd'
days = '365'
cg_api_key = 'CG-i5kd5dR33FYgvAr5jV2frxZG'
for chain in chains:
    time.sleep(1)
    chain = chain.lower()
    url = "https://api.coingecko.com/api/v3/coins/{chain}/market_chart?vs_currency={currency}&days={days}".format(chain=chain, currency=currency,days=days)
    headers = {"accept": "application/json", "x-cg-pro-api-key": cg_api_key}
    response = requests.get(url)
    
    for row in response.json()['market_caps']:
        new_row = {}
        new_row['EXTRACT_TIME'] = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")
        new_row['BLOCKCHAIN'] = chain.upper()
        new_row['DATE'] = datetime.fromtimestamp(row[0]/1000).strftime("%Y-%m-%d")
        new_row['MARKETCAP'] = int(row[1])
        marketcap_formatted_json_response += [new_row]

    for row in response.json()['total_volumes']:
        new_row = {}
        new_row['EXTRACT_TIME'] = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")
        new_row['BLOCKCHAIN'] = chain.upper()
        new_row['DATE'] = datetime.fromtimestamp(row[0]/1000).strftime("%Y-%m-%d")
        new_row['VOLUME'] = int(row[1])
        volume_formatted_json_response += [new_row]

In [ ]:
errors = client.insert_rows_json(marketcap_table_id, marketcap_formatted_json_response)

In [ ]:
if errors:
    print("Ran into errors uploading!")

In [ ]:
errors = client.insert_rows_json(volume_table_id, volume_formatted_json_response)

In [ ]:
if errors:
    print("Ran into errors uploading!")